In [448]:
import pandas as pd
import numpy as np

In [449]:
df = pd.read_csv('files/train.csv')
df = df.sample(frac = 1)

In [450]:
df.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


# Check for NaN columns..


In [451]:
for cols in df.columns:
    if df[cols].isnull().any():
        print(cols)

LotFrontage
Alley
MasVnrType
MasVnrArea
BsmtQual
BsmtCond
BsmtExposure
BsmtFinType1
BsmtFinType2
Electrical
FireplaceQu
GarageType
GarageYrBlt
GarageFinish
GarageQual
GarageCond
PoolQC
Fence
MiscFeature


# Let's go through how I might want to treat each column, before I get into dealing with NaN's...

## MSSubClass

The values it takes are kind of weird considering its categorical data. If I decide to use a decision tree-based algorithm then it's fine. If not I'll need to at least normailize this one.

## MSZoning 
Should be one-hot encoded.

## LotFrontage, LotArea
Numerical data. We're dealing with a bunch of different units here, and if I wasn't working with a decision tree I'd really need to be careful to normalize and standardize the data.

## Street, Alley, LandContour, LotConfig, Neighborhood, Condition1, Condition2, BldgType, HouseStyle
Categorical. To be one-hot encoded.

## OverallCond and OverallQual
Numerical.

## YearBuilt, YearRemodAdd
Numerical data. Need to think about how I should treat this. Could maybe base them off of how many days ago they were built / remodeled.

## RoofStyle, RoofMatl, Exterior1st, Exterior2nd, MasVnrType
All categorical.

## MasVnrArea
Numerical.

## LotShape, Utilities, LandSlope, ExterQual, ExterCond
Categorical, but admits an ordered set, with Excellent > Good > Average/Typical etc.. so can convert to numerical to save on columns

## Foundation
Categorical.

## BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1
Categorical, but again admitting an ordered set, so should be converted into numerical data to save on columns.

## BsmtFinSF1
Numerical.

## BsmtFinType2
Categorical, but admitting an ordered set -- convert to numeric.

## BsmtFinSF2, BsmtUnfSF, TotalBsmtSF
Numerical.

## Heating
Categorical.

## HeatingQC
Categorical, but ordered set so convert to numeric.

## CentralAir, Electrical
Categorical.

## 1stFlrSF, 2ndFlrSF, LowQualFinSF, GrLivArea, BsmtFullBath,  BsmtHalfBath, FullBath, HalfBath, Bedroom, Kitchen
Numerical

## KitchenQual
Categoric, but admits ordered set, so convert to numerical.

## TotRmsAbvGrd
Numerical.

## Functional
Categorical, but admits ordered set. Convert to numerical.

## Fireplaces
Numerical.

## FireplaceQu
Categorical -- convertable to numerical.

## GarageType
Categorical.

## GarageFinish
Categorical -- convertable to numerical.

## GarageYrBlt
Possibly convert to 'how many days ago'.

## GarageCars, GarageArea
Numerical.

## GarageQual, GarageCond
Categorical -- convertable to numerical.

## PavedDrive
I think this admits an ordered set in terms of 'paved-ness'. So possibly convertable to numerical, although could be safe and just keep it categorical -- only like 3 unique values.

## WoodDeckSF, OpenPorchSF, EnclosedPorch, 3SsnPorch, ScreenPorch, PoolArea
Numerical.

## PoolQC, Fence
Categorical -- convertable to numerical.

## MiscFeature
Categorical.

## MiscVal
Numerical.

## MoSold , YrSold
MoSold might not be worth the effort to include. I won't want to one-hot encode 12 columns for each month, and I doubt the month something was sold can be worth the 12 columns I'd be costing the training data. YrSold I can express in terms of 'days since the present'. I could maybe express MoSold as numerical data of values between 1 and 12. But I don't think that's justifiable because it's weird to say that month 2 > month 1 or something.

## SaleType, SaleCondition
Categorical.

# Okay, with that, here are my next steps:

There isn't a whole lot of rows to this data, so if I add too many columns we start worrying about big-p, little-n issues. I need to try and impute my NaN's instead of deleting those rows so that the little-n doesn't get even smaller. I'll also really want to favor converting categorical data that admits an ordered set into numerical data so I can save up on columns.
1. Create a function to deal with missing NaN values sample-by-sample by selecting a subset of the data that shares as many characteristics as possible with a sample and picking the most likely value to impute given this. 
2. Create a function that converts categorical-but-numerical-convertable columns into numeric columns.
3. Deal with time-series columns.

It might be worth trying to find similar columns by one-hot encoding and normalizing data, and seeing which samples have the highest cosine similarity to the one in question.. I'd need to normalize numerical data, or else different magnitudes would have disproportionate influence on the the cosine similarity..

# One-hot encoding categorical data

Also going to drop the MoSold column.

In [452]:
df = df.drop(columns = ['MoSold','Id'])

In [453]:
to_one_hot = [
    'MiscFeature',
    'SaleType',
    'SaleCondition',
    'CentralAir', 
    'Electrical',
    'Heating',
    'Foundation',
    'RoofStyle', 
    'RoofMatl', 
    'Exterior1st', 
    'Exterior2nd', 
    'MasVnrType',
    'Street', 
    'Alley', 
    'LandContour', 
    'LotConfig', 
    'Neighborhood', 
    'Condition1', 
    'Condition2', 
    'BldgType', 
    'HouseStyle',
    'MSZoning',
    'MSSubClass',
    'GarageType',
    'PavedDrive'
    ]

In [454]:
time_series_columns = ['YrSold','YearBuilt','YearRemodAdd','GarageYrBlt']

And now for the data I want to make numeric.

In [455]:
numerical = [
    'MiscVal',
    'WoodDeckSF', 
    'OpenPorchSF', 
    'EnclosedPorch', 
    '3SsnPorch', 
    'ScreenPorch', 
    'PoolArea',
    'GarageCars', 
    'GarageArea',
    'Fireplaces',
    'TotRmsAbvGrd',
    '1stFlrSF', 
    '2ndFlrSF', 
    'LowQualFinSF', 
    'GrLivArea', 
    'BsmtFullBath',  
    'BsmtHalfBath', 
    'FullBath', 
    'HalfBath', 
    'BedroomAbvGr', 
    'KitchenAbvGr',
    'BsmtFinSF2', 
    'BsmtUnfSF', 
    'TotalBsmtSF',
    'BsmtFinSF1',
    'MasVnrArea',
    'LotFrontage', 
    'LotArea',
    'OverallCond',
    'OverallQual',
      
]

In [456]:
columns_so_far = to_one_hot + time_series_columns + numerical

In [457]:
full_columns = list(df.columns)

In [458]:
to_numerical = list(set(columns_so_far).symmetric_difference(full_columns))

In [459]:
to_numerical.remove('SalePrice')

In [460]:
to_numerical

['GarageFinish',
 'Functional',
 'GarageCond',
 'HeatingQC',
 'BsmtCond',
 'PoolQC',
 'KitchenQual',
 'Utilities',
 'ExterCond',
 'BsmtExposure',
 'ExterQual',
 'GarageQual',
 'Fence',
 'LotShape',
 'FireplaceQu',
 'LandSlope',
 'BsmtFinType1',
 'BsmtFinType2',
 'BsmtQual']

## One-hot encode the categorical columns

In [461]:
df = pd.get_dummies(df,columns = to_one_hot)

## Convert the to_numerical data to numerical

In [462]:
import json 
with open('files/to_numerical_json.json', 'r') as f:   
    ordered_categories = json.load(f)

In [463]:
ordered_categories

{'Fence': ['GdPrv', 'MnPrv', 'GdWo', 'MnWw', 'NA'],
 'LotShape': ['Reg', 'IR1', 'IR2', 'IR3'],
 'Utilities': ['AllPub', 'NoSewr', 'NoSeWa', 'ELO'],
 'LandSlope': ['Gtl', 'Mod', 'Sev'],
 'BsmtQual': ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'NA'],
 'BsmtCond': ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'NA'],
 'BsmtExposure': ['Gd', 'Av', 'Mn', 'No', 'NA'],
 'BsmtFinType1': ['GLQ', 'ALQ', 'BLQ', 'Rec', 'LwQ', 'Unf', 'NA'],
 'BsmtFinType2': ['GLQ', 'ALQ', 'BLQ', 'Rec', 'LwQ', 'Unf', 'NA'],
 'HeatingQC': ['Ex', 'Gd', 'TA', 'Fa', 'Po'],
 'KitchenQual': ['Ex', 'Gd', 'TA', 'Fa', 'Po'],
 'Functional': ['Typ', 'Min1', 'Min2', 'Mod', 'Maj1', 'Maj2', 'Sev', 'Sal'],
 'FireplaceQu': ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'NA'],
 'GarageFinish': ['Fin', 'RFn', 'Unf', 'NA'],
 'ExterCond': ['Ex', 'Gd', 'TA', 'Fa', 'Po'],
 'ExterQual': ['Ex', 'Gd', 'TA', 'Fa', 'Po'],
 'GarageQual': ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'NA'],
 'GarageCond': ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'NA'],
 'PoolQC': ['Ex', 'Gd', 'TA', 'Fa', 'NA']}

In [464]:
def conv_to_numerical(df, ordered_categories):
    #print(ordered_categories.keys())
    for col in ordered_categories.keys():
        #print(col)
        values = ordered_categories[col]
        ordering = list(reversed(range(len(values))))
        mappings = dict(list(zip(values, ordering)))
        #print(mappings)
        #print(df[col])
        for index, row in df.iterrows():
            x = row[col]
            if not isinstance(x, float):
                df.loc[index, col] = mappings[x]
        #print(df[col])
    return df

In [465]:
conv_to_numerical(df, ordered_categories)

Fence
{'GdPrv': 4, 'MnPrv': 3, 'GdWo': 2, 'MnWw': 1, 'NA': 0}
295     GdPrv
1413      NaN
919       NaN
215     MnPrv
1172      NaN
        ...  
43      MnPrv
792       NaN
515       NaN
957       NaN
393      GdWo
Name: Fence, Length: 1460, dtype: object
295       4
1413    NaN
919     NaN
215       3
1172    NaN
       ... 
43        3
792     NaN
515     NaN
957     NaN
393       2
Name: Fence, Length: 1460, dtype: object
LotShape
{'Reg': 3, 'IR1': 2, 'IR2': 1, 'IR3': 0}
295     IR1
1413    IR1
919     IR1
215     IR1
1172    IR1
       ... 
43      IR1
792     IR1
515     Reg
957     Reg
393     Reg
Name: LotShape, Length: 1460, dtype: object
295     2
1413    2
919     2
215     2
1172    2
       ..
43      2
792     2
515     3
957     3
393     3
Name: LotShape, Length: 1460, dtype: object
Utilities
{'AllPub': 3, 'NoSewr': 2, 'NoSeWa': 1, 'ELO': 0}
295     AllPub
1413    AllPub
919     AllPub
215     AllPub
1172    AllPub
         ...  
43      AllPub
792     AllPub
515     Al

,LotFrontage,LotArea,LotShape,Utilities,LandSlope,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,...,MSSubClass_190,GarageType_2Types,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,PavedDrive_N,PavedDrive_P,PavedDrive_Y
295,37.0,7937,2,3,2,6,6,1984,1984,0.0,...,0,0,0,0,0,0,1,0,0,1
1413,88.0,10994,2,3,2,8,5,2005,2006,366.0,...,0,0,1,0,0,0,0,0,0,1
919,87.0,11029,2,3,2,6,8,1958,2002,0.0,...,0,0,1,0,0,0,0,0,0,1
215,72.0,10011,2,3,2,5,6,1957,1996,64.0,...,0,0,1,0,0,0,0,0,0,1
1172,35.0,4017,2,3,2,7,5,2006,2007,0.0,...,0,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43,NaN,9200,2,3,2,5,6,1975,1980,0.0,...,0,0,0,0,0,0,1,0,0,1
792,92.0,9920,2,3,2,7,5,1996,1997,0.0,...,0,0,1,0,0,0,0,0,0,1
515,94.0,12220,3,3,2,10,5,2009,2009,305.0,...,0,0,1,0,0,0,0,0,0,1
957,70.0,7420,3,3,2,5,5,1962,1962,0.0,...,0,0,0,0,0,0,1,0,0,1


# Normalize the numerical columns for the cosine similarity steps
Normalizing is not generally a good idea for a decision tree-based model, so I'll probably make it separate to df.

In [466]:
from sklearn import preprocessing

min_max_scaler = preprocessing.MinMaxScaler()

cos_sim_df = df.copy()
#cos_sim_df = cos_sim_df.drop(columns = numerical)
cos_sim_df[numerical] = min_max_scaler.fit_transform(df[numerical])

In [467]:
cos_sim_df

,LotFrontage,LotArea,LotShape,Utilities,LandSlope,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,...,MSSubClass_190,GarageType_2Types,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,PavedDrive_N,PavedDrive_P,PavedDrive_Y
295,0.054795,0.031022,2,3,2,0.555556,0.625,1984,1984,0.000000,...,0,0,0,0,0,0,1,0,0,1
1413,0.229452,0.045311,2,3,2,0.777778,0.500,2005,2006,0.228750,...,0,0,1,0,0,0,0,0,0,1
919,0.226027,0.045474,2,3,2,0.555556,0.875,1958,2002,0.000000,...,0,0,1,0,0,0,0,0,0,1
215,0.174658,0.040716,2,3,2,0.444444,0.625,1957,1996,0.040000,...,0,0,1,0,0,0,0,0,0,1
1172,0.047945,0.012700,2,3,2,0.666667,0.500,2006,2007,0.000000,...,0,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43,NaN,0.036925,2,3,2,0.444444,0.625,1975,1980,0.000000,...,0,0,0,0,0,0,1,0,0,1
792,0.243151,0.040291,2,3,2,0.666667,0.500,1996,1997,0.000000,...,0,0,1,0,0,0,0,0,0,1
515,0.250000,0.051041,3,3,2,1.000000,0.500,2009,2009,0.190625,...,0,0,1,0,0,0,0,0,0,1
957,0.167808,0.028605,3,3,2,0.444444,0.500,1962,1962,0.000000,...,0,0,0,0,0,0,1,0,0,1


In [468]:
def smart_fillna(column):
    

SyntaxError: unexpected EOF while parsing (800286481.py, line 2)